<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Lab 03 - Introduction To Data Science @ FIT-HCMUS, VNU-HCM
</div>

<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 01 - Data collecting
</div>

In [1]:
import re
import random
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [2]:
url = 'https://www.coursera.org/sitemap~www~courses.xml'
root = ET.fromstring(requests.get(url).content)
links = []
for child in root:
    for data in child:
        links.append(data.text)
        
file_path = 'course_url.txt'
with open(file_path, 'w') as txt_file:
    for item in links:
        txt_file.write(str(item) + '\n')
print(f'Data (with {len(links)} links) has been written to {file_path}')

Data (with 8001 links) has been written to course_url.txt


In [3]:
name_list = []
field_list = []
enrollment_list = []
language_list = []
rating_list = []
level_list = []
duration_list = []
description_list = []
skill_list = []
instructor_list = []
offered_list = []

with open('course_url.txt', 'r') as file:
    for line in file:
        url = line.strip()
        print(url)
        try:
            content = requests.get(url, timeout=30)
        except requests.exceptions.ConnectionError:
            continue
        
        soup = BeautifulSoup(content.content, "html.parser")

        name = soup.find('h1', attrs={'class':'cds-119 cds-Typography-base css-1xy8ceb cds-121'})
        name = name.text if name is not None else 'None'

        fields = soup.find_all('li', attrs={'class':'cds-breadcrumbs-listItem'})
        field = ' > '.join(item.text for item in fields[2:] if item)
        
        enrollment = soup.find_all('div', attrs={'class':'css-kd6yq1'})
        if len(enrollment) == 0:
            enrollment = soup.find_all('div', attrs={'class':'css-6chzbh'})
            if len(enrollment) == 0:
                continue
            else:
                enrollment = enrollment[1]
        else:
            enrollment = enrollment[1]
        enrollment = enrollment.text if enrollment is not None else 'None'

        language = soup.find_all('p', attrs={'class':'cds-119 cds-Typography-base css-80vnnb cds-121'})
        language = next((item.text for item in language if 'Taught in' in item.text), 'None')

        detail = soup.find_all('div', attrs={'class':'cds-119 cds-Typography-base css-h1jogs cds-121'})
        if len(detail) == 0:
            continue
        rating = ''
        level = ''
        duration = ''
        for item in detail:
            if item:
                content = item.text
                pattern = re.compile(r'\b\d+\.\d+\b')
                match = pattern.search(content)
                rating = match.group() if match else rating
                level = content if 'level' in content else level
                duration = content if 'hour' in content else duration

        about = soup.find('div', attrs={'id':'about', 'class':'css-qpssb7'})
        if about is None:
            continue
        description = about.find('ul', attrs={'class':'cds-9 css-7avemv cds-10'})
        description = description.text if description is not None else 'None'
        skills = about.find_all('span', attrs={'class':'css-1l1jvyr'})
        skill = ', '.join(item.text for item in skills if item)

        info = soup.find('div', attrs={'class':'cds-9 css-0 cds-11 cds-grid-item cds-56 cds-77 cds-90'})
        if info is None:
            continue
        instructor = info.find_all('span', attrs={'class':'cds-119 cds-Typography-base css-e7lgfl cds-121'})
        if len(instructor) == 0:
            continue
        offered = instructor[-1]
        offered = offered.text if offered is not None else 'None'
        instructor = [item.text for item in instructor[:-1] if item.text is not None]
        instructor_rating = info.find('span', attrs={'class':'css-1tdi49m'})
        instructor_rating = instructor_rating.text if instructor_rating is not None else 'None'
        instructor = f'{instructor_rating} {set(instructor)}'
        
        name_list.append(name)
        field_list.append(field)
        enrollment_list.append(enrollment)
        language_list.append(language)
        rating_list.append(rating)
        level_list.append(level)
        duration_list.append(duration)
        description_list.append(description)
        skill_list.append(skill)
        instructor_list.append(instructor)
        offered_list.append(offered)

https://www.coursera.org/learn/ibm-cyber-threat-intelligence?specialization=security-analyst-fundamentals


In [4]:
courses = pd.DataFrame({'name': name_list,
                    'field': field_list,
                    'enrollment': enrollment_list,
                    'language': language_list,
                    'rating': rating_list,
                    'level': level_list,
                    'duration': duration_list,
                    'description': description_list,
                    'skill': skill_list,
                    'instructor': instructor_list,
                    'offered by': offered_list})

courses.to_csv('../Data/raw_data.csv', index=False)

In [5]:
courses

,name,field,enrollment,language,rating,level,duration,description,skill,instructor,offered by
0,Cyber Threat Intelligence,Information Technology > Security,"52,299 already enrolled",Taught in English,4.7,Beginner level,25 hours (approximately),None,"Cybersecurity, network defensive tactics, thre...",4.5 {'IBM Skills Network Team'},IBM
